In [1]:
import pandas as pd
import torch
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
%config Completer.use_jedi = False
import warnings
warnings.simplefilter(action='ignore')
from sklearn import preprocessing
sns.set(style="darkgrid", color_codes=True)

* loading the data...

In [1]:
df=pd.read_csv('../input/titanic/train.csv')
df

* just need to get rid of all the upper case latters for ease in the headers

In [1]:
df.drop(columns=['Ticket',],inplace=True)
df.columns=['passengerid','survived','pclass','name','sex','age','sibsp','parch','fare','cabin','embarked']

* Need to know howmany null values are there in each columns of the data, so that we can replace it by apropriate values 

* What i think is i'll just see the percent of null values in all of the column , if any of e'm has a very high percent of null values we'll drop and maybe explore a bit later! ,rest of e'm we'll fill with the mean value




In [1]:
null=df.isnull().sum()# entire
percent=null/len(df)*100
print(percent)
percent=percent[percent>0]
sns.barplot(x=percent.index,y=percent)
plt.show()

In [1]:
print(df['embarked'].nunique())
print(df['embarked'].unique())


* All the string values to be changed to int if its that sort of a parameter
* Here embarked has 3 values we'll change it to 1,2,3 correspondingly
* Sex to 1 & 0, the same way
* The missing values in both categories can be changed to its avarage or what the most of the data tend towards

In [1]:
df['embarked-s']=np.zeros(len(df))
df['embarked-c']=np.zeros(len(df))
df['embarked-q']=np.zeros(len(df))
sns.countplot(df['embarked'])
plt.show()


* We can makes separate columns for different embarked values

In [1]:
# from the graph its fair assume the missing values to embarked=S
df['embarked'].fillna('S',inplace=True)
df.loc[df['embarked']=='S','embarked-s']=1
df.loc[df['embarked']=='C','embarked-c']=1
df.loc[df['embarked']=='Q','embarked-q']=1



* nan values of age replaced by the mean value

In [1]:
df.loc[df['sex']=='male','sex']=1
df.loc[df['sex']=='female','sex']=0
df['age'].fillna(df['age'].mean(),inplace=True)

In [1]:
df['cabin'].nunique()
# df['cabin'].unique()

In [1]:
df['owned_cabin']=df['cabin'].isnull().astype(int)
df['is_minor']=np.zeros(len(df))
df.loc[df['age']<18,'is_minor']=1

df['pclass_1']=np.zeros(len(df))
df['pclass_2']=np.zeros(len(df))
df['pclass_3']=np.zeros(len(df))
df.loc[df['pclass']==1,'pclass_1']=1
df.loc[df['pclass']==2,'pclass_2']=1
df.loc[df['pclass']==3,'pclass_3']=1


# df.drop('cabin',axis=1,inplace=True)
# df.drop('name',axis=1,inplace=True)

In [1]:
df.describe()

# # EDA
Lets see what insights can we get from the graph ..


In [1]:
sns.countplot(data=df,x='survived',hue='sex')
plt.show()

#oh my god i did't know that

* most of the survivers were sex=0, also the death rate for sex=0 is very low compared to the other one


In [1]:
sns.set(style='darkgrid')
sns.countplot(df['pclass'],hue=df['survived'])
sns.jointplot(df['pclass'],df['sex'],kind='hex',color='pink')
sns.jointplot(df['pclass'],df['age'],hue=df['survived'])
sns.jointplot(df['pclass'],df['age'],hue=df['sex'])
sns.jointplot(df['pclass'],df['age'],hue=df['embarked'],kind='kde')
sns.jointplot(df['survived'],df['sex'],hue=df['embarked'],kind='kde')
plt.show()
# sns.distplot(df['age'],ax=ax[0][1],kde=True,)

In [1]:
sns.jointplot(df['pclass'],df['sibsp'],kind='hex',color='green')
sns.jointplot(df['pclass'],df['parch'],kind='hex',color='yellow')
plt.show()

In [1]:
sns.jointplot(df['pclass'],df['fare'],hue=df['survived'],palette='rocket')
sns.jointplot(df['sex'],df['fare'],hue=df['survived'],kind='kde',palette ='rocket')

In [1]:
# sns.pairplot(df,hue='survived',palette='rocket',kind='kde')


In [1]:
corr_matrix=df.corr()
figure=plt.figure(figsize=(15,10))
sns.heatmap(corr_matrix,annot=True,cmap='Blues')
plt.show()

 # Model building and testing

importing bunch of things  for different algorithms

In [1]:
from sklearn.model_selection import train_test_split,cross_val_score,GridSearchCV
from sklearn.metrics import accuracy_score,confusion_matrix,classification_report
from sklearn.svm import LinearSVC,SVC
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler
import torch
import torch.nn as nn

In [1]:
df.drop(columns={'pclass','name','cabin','embarked'},inplace=True)
df.head()

the data is already prepared, and we need separate sets of them for training and validation


In [1]:
X=df.drop(columns={'survived','is_minor'}, axis=1)
Y=df['survived']
train_x,validation_x,train_y,validation_y=train_test_split(X,Y.to_numpy(),test_size=0.2,random_state=10)
 

Lets try applying a Linear classifier

In [1]:
clf=LinearSVC(C=0.1,random_state=0)
clf.fit(train_x,train_y)
y_preds=clf.predict(validation_x)
accuracy=accuracy_score(validation_y,y_preds)
print(accuracy)

The confusion matrix shows how have the model really made the predictions, where all the currect classification represented along the diagonal, and all the misclassified, and how they have been misclssified is given as the other elements of the matrix.(if a positive sample is classified -ve or all the other combinations of potential errors.)

In [1]:
confusion_matrix(validation_y,y_preds)


In [1]:
param_grid = {'C': 10. ** np.arange(-10, 4)}
grid_search = GridSearchCV(clf, param_grid=param_grid, cv=6,)
grid_search.fit(train_x,train_y)


Here is  a parameter search with gridsearch, from sklearn.modelselection. As mensioned earlier the parameter c is really vital here since it controlls the distance from the descision to the margin boundary. What the grid search doea is, we can give it an array of different c values,the algorithm takes the values and cross valuidate through the entire data set and outputs the accuracies for each of the cross validated piece of data, then the avarage of the accuracies is taken as the representing accuracy for the perticular value of the parameter. 

In [1]:
print(grid_search.best_params_)
print(grid_search.best_score_)


now using the best parameter from grid search to train the data and check on the validation set

In [1]:
res_data=pd.DataFrame(grid_search.cv_results_)
res_data

In [1]:
plt.figure(figsize=(8,5))
plt.plot(res_data['param_C'],res_data['mean_test_score'])
plt.xticks([0,100,1000],rotation=90)
plt.show()

In [1]:
y_preds = grid_search.predict(validation_x)
print('Accuracy: %.10f' % accuracy_score(validation_y, y_preds))

**Lets try applying logistic regression**

In [1]:
#zero mean and unit variance
sc=StandardScaler()
train_x=sc.fit_transform(train_x)
validation_x=sc.transform(validation_x)

In [1]:

lr_clf = LogisticRegression(solver='liblinear')
lr_clf.fit(train_x, train_y)
pred = lr_clf.predict(train_x)
clf_report = pd.DataFrame(classification_report(train_y, pred, output_dict=True))
print(clf_report)
print(accuracy_score(train_y, pred) )
    

In [1]:
pred = lr_clf.predict(validation_x)
clf_report = pd.DataFrame(classification_report(validation_y, pred, output_dict=True))
print(accuracy_score(validation_y, pred) )

In [1]:
#preparing test_data 

test_data=pd.read_csv('../input/titanic/test.csv')

test_data['pclass_1']=np.zeros(len(test_data))
test_data['pclass_2']=np.zeros(len(test_data))
test_data['pclass_3']=np.zeros(len(test_data))
test_data.loc[test_data['Pclass']==1,'pclass_1']=1
test_data.loc[test_data['Pclass']==2,'pclass_2']=1
test_data.loc[test_data['Pclass']==3,'pclass_3']=1

test_data['embarked-s']=np.zeros(len(test_data))
test_data['embarked-c']=np.zeros(len(test_data))
test_data['embarked-q']=np.zeros(len(test_data))

test_data['Embarked'].fillna('S',inplace=True)
test_data.loc[test_data['Embarked']=='S','embarked-s']=1
test_data.loc[test_data['Embarked']=='C','embarked-c']=1
test_data.loc[test_data['Embarked']=='Q','embarked-q']=1

test_data['owned_cabin']=test_data['Cabin'].isnull().astype(int)
test_data['is_minor']=np.zeros(len(test_data))
test_data.loc[test_data['Age']<18,'is_minor']=1

test_data.loc[test_data['Sex']=='male','Sex']=1
test_data.loc[test_data['Sex']=='female','Sex']=0

test_data['Age'].fillna(test_data['Age'].mean(),inplace=True)
test_data['Fare'].fillna(test_data['Fare'].mean(),inplace=True)

test_data.drop(columns={'Name','Ticket','Cabin','Embarked','Pclass','is_minor'},inplace=True)



0.8324 seems the maximum accuracy logi regression can output given the feature representations

In [1]:
Survived=lr_clf.predict(test_data)
file=pd.DataFrame(test_data['PassengerId'])
file['Survived']=Survived

In [1]:
file.to_csv('submission.csv', header=True, index=False)